<a href="https://colab.research.google.com/github/venura449/Brain-Tumor-Detection-System/blob/main/Brain_Tumor_detection_through_MRI_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
navoneel_brain_mri_images_for_brain_tumor_detection_path = kagglehub.dataset_download('navoneel/brain-mri-images-for-brain-tumor-detection')

print('Data source import complete.')


Data source import complete.


In [2]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import os
from PIL import Image

def generate_labels(image_dir, label_dir, class_id):
    os.makedirs(label_dir, exist_ok=True)
    for fname in os.listdir(image_dir):
        if not fname.lower().endswith(".jpg"):
            continue
        img_path = os.path.join(image_dir, fname)
        label_path = os.path.join(label_dir, fname.replace(".jpg", ".txt"))

        with Image.open(img_path) as img:
            w, h = img.size
            # Generate a center bounding box (50% size as example)
            xc, yc, bw, bh = 0.5, 0.5, 0.5, 0.5
            with open(label_path, "w") as f:
                f.write(f"{class_id} {xc} {yc} {bw} {bh}\n")

generate_labels('/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes', 'labels/train', class_id=0)


In [4]:
yaml_content = """
path: ./dataset
train: images/train
val: images/val
nc: 1
names: ['tumor']
"""

with open("dataset.yaml", "w") as f:
    f.write(yaml_content)


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Source folders
yes_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes"
no_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no"

# Create YOLO-style folders
base = "/kaggle/working/dataset"
img_train_dir = f"{base}/images/train"
img_val_dir = f"{base}/images/val"
label_train_dir = f"{base}/labels/train"
label_val_dir = f"{base}/labels/val"

for d in [img_train_dir, img_val_dir, label_train_dir, label_val_dir]:
    os.makedirs(d, exist_ok=True)


In [6]:
from PIL import Image

def create_label_file(img_path, label_path, class_id=0):
    with Image.open(img_path) as img:
        w, h = img.size
        # Dummy bounding box in center
        xc, yc, bw, bh = 0.5, 0.5, 0.6, 0.6
        with open(label_path, "w") as f:
            f.write(f"{class_id} {xc} {yc} {bw} {bh}\n")

# Get all images
yes_imgs = [os.path.join(yes_dir, f) for f in os.listdir(yes_dir) if f.endswith(".jpg")]
no_imgs = [os.path.join(no_dir, f) for f in os.listdir(no_dir) if f.endswith(".jpg")]

# Only use tumor images for detection
train_imgs, val_imgs = train_test_split(yes_imgs, test_size=0.2, random_state=42)

# Copy & label training images
for src in train_imgs:
    fname = os.path.basename(src)
    dst = os.path.join(img_train_dir, fname)
    shutil.copy(src, dst)
    create_label_file(dst, os.path.join(label_train_dir, fname.replace(".jpg", ".txt")))

# Copy & label validation images
for src in val_imgs:
    fname = os.path.basename(src)
    dst = os.path.join(img_val_dir, fname)
    shutil.copy(src, dst)
    create_label_file(dst, os.path.join(label_val_dir, fname.replace(".jpg", ".txt")))


In [7]:
yaml_content = """
path: /kaggle/working/dataset
train: images/train
val: images/val
nc: 1
names: ['tumor']
"""

with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(yaml_content)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # nano model

model.train(
    data="/kaggle/working/dataset.yaml",
    epochs=10,
    imgsz=640,
    batch=8,
    name="brain_tumor_yolo",
    workers=2,
    device="cpu"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 72.3MB/s]


Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=brain_tumor_yolo, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pret

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/10         0G      1.277      1.999      1.928          4        640: 100%|██████████| 9/9 [00:53<00:00,  5.96s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.17s/it]

                   all         18         18          1      0.694      0.995      0.728



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/10         0G      1.151       1.92      1.704          8        640:  56%|█████▌    | 5/9 [00:30<00:23,  5.98s/it]

In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/detect/brain_tumor_yolo/weights/best.pt")

# Predict on an image and show results directly
results = model.predict("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/Y1.jpg", imgsz=640, conf=0.25, device="cpu")

# Show annotated image in notebook (inline display)
results[0].show()


In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/detect/brain_tumor_yolo/weights/best.pt")

# Predict on an image and show results directly
results = model.predict("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/Y74.jpg", imgsz=640, conf=0.25, device="cpu")

# Show annotated image in notebook (inline display)
results[0].show()


In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/detect/brain_tumor_yolo/weights/best.pt")

# Predict on an image and show results directly
results = model.predict("/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/Y11.jpg", imgsz=640, conf=0.25, device="cpu")

# Show annotated image in notebook (inline display)
results[0].show()

In [ ]:
# prompt: check a results without tumor images

# Load your trained model
model = YOLO("runs/detect/brain_tumor_yolo/weights/best.pt")

# Predict on an image from the 'no' directory (no tumor) and show results directly
results = model.predict("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 100.jpg", imgsz=640, conf=0.25, device="cpu")

# Show annotated image in notebook (inline display)
results[0].show()

# Predict on another image from the 'no' directory
results = model.predict("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 3.jpg", imgsz=640, conf=0.25, device="cpu")

# Show annotated image in notebook (inline display)
results[0].show()

# Predict on yet another image from the 'no' directory
results = model.predict("/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/no 923.jpg", imgsz=640, conf=0.25, device="cpu")

# Show annotated image in notebook (inline display)
results[0].show()